# QUICKSTART
本节贯穿机器学习中常见任务的 API。请参阅每个部分中的链接以深入了解。

## Working with data
PyTorch 有两个用于[处理数据的原语](https://pytorch.org/docs/stable/data.html)：torch.utils.data.DataLoader 和 torch.utils.data.Dataset。Dataset 存储样本及其相应的标签，DataLoader 将一个 iterable 包装在 Dataset 里面。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch 提供特定领域的库，例如 [TorchText](https://pytorch.org/text/stable/index.html)、[TorchVision](https://pytorch.org/vision/stable/index.html) 和 [TorchAudio](https://pytorch.org/audio/stable/index.html)，所有这些库都包含数据集。对于本教程，我们将使用 TorchVision 数据集。

torchvision.datasets 模块包含许多现实世界视觉数据的 Dataset 对象，如 CIFAR、COCO（此处为[完整列表](https://pytorch.org/vision/stable/datasets.html)）。在本教程中，我们使用 FashionMNIST（翻译者注：FashionMNIST 是一个替代 MNIST 手写数字集的图像数据集。 其涵盖了来自 10 种类别的共 7 万个不同商品的正面图片。FashionMNIST 的大小、格式和训练集/测试集划分与原始的 MNIST 完全一致。60000/10000 的训练测试数据划分，28x28 的灰度图片） 数据集。

每个 TorchVision 数据集都包含两个参数：transform 和 target_transform，分别用于修改样本和标签。

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor(),
)

我们将数据集作为参数传递给 DataLoader。这在我们的数据集上包装了一个可迭代对象，并支持自动批处理、采样、混洗和多进程数据加载。这里我们定义了一个 64 的批量大小，即数据加载器迭代器中的每个元素将返回一批 64 个特征和标签。

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


阅读有关在 PyTorch 中[加载数据](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)的更多信息。

## 创建模型
为了在 PyTorch 中定义神经网络，我们创建了一个继承自 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 的类。我们在 __init__ 函数中定义网络层，并在 forward 函数中指定数据在网络中如何进行前向传播。为了加速神经网络的训练，我们将其移至 GPU 或 MPS（如果可用）。

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


阅读有关在 PyTorch 中[构建神经网络](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)的更多信息。

## 优化模型参数
要训练模型，我们需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)和一个[优化器](https://pytorch.org/docs/stable/optim.html)。

In [7]:
loos_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型对训练数据集（分批输入）进行预测，并反向传播预测误差以调整模型的参数。

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还根据测试数据集检查模型的性能，以确保它正在学习。

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程是在几个迭代（时期）进行的。在每个迭代内，模型都会学习参数以做出更好的预测。我们在每个时期都打印模型的准确性和损失；我们希望看到每个时期的准确性提高和损失下降。

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loos_fn, optimizer)
    test(test_dataloader, model, loos_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315501  [   64/60000]
loss: 2.310323  [  704/60000]
loss: 2.314732  [ 1344/60000]
loss: 2.299647  [ 1984/60000]
loss: 2.299818  [ 2624/60000]
loss: 2.304317  [ 3264/60000]
loss: 2.297630  [ 3904/60000]
loss: 2.305325  [ 4544/60000]
loss: 2.305777  [ 5184/60000]
loss: 2.291766  [ 5824/60000]
loss: 2.306197  [ 6464/60000]
loss: 2.291735  [ 7104/60000]
loss: 2.295366  [ 7744/60000]
loss: 2.291661  [ 8384/60000]
loss: 2.292646  [ 9024/60000]
loss: 2.285110  [ 9664/60000]
loss: 2.295316  [10304/60000]
loss: 2.282382  [10944/60000]
loss: 2.293025  [11584/60000]
loss: 2.282354  [12224/60000]
loss: 2.287026  [12864/60000]
loss: 2.286879  [13504/60000]
loss: 2.284072  [14144/60000]
loss: 2.280036  [14784/60000]
loss: 2.286460  [15424/60000]
loss: 2.277164  [16064/60000]
loss: 2.271809  [16704/60000]
loss: 2.274873  [17344/60000]
loss: 2.276842  [17984/60000]
loss: 2.278392  [18624/60000]
loss: 2.275007  [19264/60000]
loss: 2.266371  [19904/60000]


阅读有关[训练模型](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)的更多信息。

## 保存模型
保存模型的一种常见方法是序列化内部状态字典（包含模型参数）。

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

现在，该模型可以用于做出预测。

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


阅读有关[保存和加载模型](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)的更多信息。